In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils,print_summary,to_categorical
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K


              
    




Using TensorFlow backend.


In [2]:
data= pd.read_csv(r'C:\Users\Admin\jupyter projects\devnagri.csv',header=0)
dataset= np.array(data)
x=dataset
y=dataset
x=x[:,0:1024]
y=y[:,1024]

#splitting the data into training and testing set, normalizing the values
x_train=x[0:71000,:]    # 71000 samples in training set
x_train=x_train/255     # coverting the pixel values0-255 into 0-1
x_test=x[71000:72001,:] # 1000 samples in testing set
x_test=x_test/255






              

In [3]:
# reshaping the y values to ensure correct dimensionality
y=y.reshape(y.shape[0],1)
y_train=y[0:71000,:]
y_train=y_train.T
y_test=y[71000:72001,:]
y_test=y_test.T

# to check dimensionality of test and train data
print("no. of training examples:"+str(x_train.shape[0]))
print("no. of test examples:"+str(x_test.shape[0]))
print("x_train shape:"+ str(x_train.shape))
print("x_test shape:"+ str(x_test.shape))
print("y_train shape:"+str(y_train.shape))
print("y_test shape: "+str(y_test.shape))

image_x,image_y=32,32   # defining the dimension of image
train_y=np_utils.to_categorical(y_train)  # converting integers values to binary values
test_y=np_utils.to_categorical(y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
x_train=x_train.reshape(x_train.shape[0],image_x,image_y,1) # 1 is for gray image
x_test=x_test.reshape(x_test.shape[0],image_x,image_y,1)
print("x_train shape:"+ str(x_train.shape))
print("y_train shape:"+ str(train_y.shape))

no. of training examples:71000
no. of test examples:1000
x_train shape:(71000, 1024)
x_test shape:(1000, 1024)
y_train shape:(1, 71000)
y_test shape: (1, 1000)
x_train shape:(71000, 32, 32, 1)
y_train shape:(71000, 37)


In [4]:
# Building a model
def keras_model(image_x,image_y): # height and width of image
    n_classes=37    # from y_train shape
    model=Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu',input_shape=(image_x,image_y,1)))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(5,5),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(n_classes,activation="softmax"))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devnagri.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
              
    return model,callbacks_list            

model,callbacks_list=keras_model(image_x,image_y)
model.fit(x_train,train_y,validation_data=(x_test,test_y),epochs=3,batch_size=64,callbacks=callbacks_list)
score=model.evaluate(x_test,test_y,verbose=0)
print("CNN ERROR: %.2f%%" % (100-score[1]*100))
print_summary(model)
model.save("devnagri.h5")

W0927 20:41:53.067623   368 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0927 20:41:53.855192   368 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0927 20:41:54.058294   368 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0927 20:41:54.397641   368 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0927 20:41:54.720563   368 dep

Train on 71000 samples, validate on 1000 samples
Epoch 1/3
71000/71000 [==============================] - 65s 922us/step - loss: 1.0920 - acc: 0.6939 - val_loss: 1.1442 - val_acc: 0.6230

Epoch 00001: val_acc improved from -inf to 0.62300, saving model to devnagri.h5
Epoch 2/3
71000/71000 [==============================] - 25s 351us/step - loss: 0.4051 - acc: 0.8802 - val_loss: 2.9462 - val_acc: 0.2630

Epoch 00002: val_acc did not improve from 0.62300
Epoch 3/3
71000/71000 [==============================] - 26s 359us/step - loss: 0.2869 - acc: 0.9135 - val_loss: 0.9086 - val_acc: 0.7110

Epoch 00003: val_acc improved from 0.62300 to 0.71100, saving model to devnagri.h5
CNN ERROR: 28.90%
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 

In [2]:
#Application:
import cv2
from keras.models import load_model
from collections import deque

model1=load_model('devnagri.h5')
print(model1)

letter_count = {0: 'CHECK', 1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna', 6: 'character_06_cha',
                7: '07_chha', 8: '08_ja', 9: '09_jha', 10: '10_yna',
                11: '11_taamatar',
                12: '12_thaa', 13: '13_daa', 14: '14_dhaa', 15: '15_adna', 16: '16_tabala', 17: '17_tha',
                18: '18_da', 19: '19_dha', 20: '20_na', 21: '21_pa', 22: '22_pha',
                23: '23_ba', 24: '24_bha', 25: '25_ma', 26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
                31: '31_petchiryakha',32: '32_patalosaw', 33: '33_ha',
                34: '34_chhya', 35: '35_tra', 36: '36_gya', 37: 'CHECK'}

def keras_predict(model, image):
    processed = keras_process_image(image)
    print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 32
    image_y = 32
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img


W1130 21:08:38.795664  5048 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1130 21:08:39.049844  5048 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1130 21:08:39.264998  5048 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1130 21:08:39.974500  5048 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1130 21:08:39.997515  504

In [5]:
import cv2
import numpy as np
try:
    cap = cv2.VideoCapture(0)
    vid_cod = cv2.VideoWriter_fourcc('M','J','P','G')
    output = cv2.VideoWriter("cam_video.avi", vid_cod, 20.0, (640,480))
    Lower_blue = np.array([110, 50, 50])
    Upper_blue = np.array([130, 255, 255])
    pred_class=0
    pts = deque(maxlen=512)
    blackboard = np.zeros((480, 640, 3), dtype=np.uint8)       # will return zero matrix of shape 480*640
    digit = np.zeros((200, 200, 3), dtype=np.uint8)  #will return zero matrix of shape 200*200
    while (cap.isOpened()):                  #while camera is opened
         ret, img = cap.read()     # reading the image
         img = cv2.flip(img, 1)   #flipping the image
         imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)    #coverting the image from RGB format to HSV 
         mask = cv2.inRange(imgHSV, Lower_blue, Upper_blue)  # defining the range where mask is applied on HSV image
         blur = cv2.medianBlur(mask, 15) # blurring the unrequired points using median blur on mask image
         blur = cv2.GaussianBlur(blur, (5, 5), 0) #blurring using gaussian blur
         thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] # defining the threshold values
         cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1] # defining the contours for getting the required feature
         # extracting blue colour from countour
         center = None
         if len(cnts) >= 1:   # finding if contour is available
            contour = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(contour) > 250:  #if contour area is greater than 250 
                ((x, y), radius) = cv2.minEnclosingCircle(contour) # enclosing it in circle
                cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(img, center, 5, (0, 0, 255), -1)
                M = cv2.moments(contour) # to calculate moments of contour
                center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00'])) # deciding the center point
                pts.appendleft(center) # appending the center point in a list
                for i in range(1, len(pts)):   
                    if pts[i - 1] is None or pts[i] is None:
                        continue
                    cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 10)
                    cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), 5)
         elif len(cnts) == 0:
            if len(pts) != []:
                blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY) # convert it into gray colour
                blur1 = cv2.medianBlur(blackboard_gray, 15)
                blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
                thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
                if len(blackboard_cnts) >= 1:
                     cnt = max(blackboard_cnts, key=cv2.contourArea)
                     print(cv2.contourArea(cnt))
                     if cv2.contourArea(cnt) > 2000:
                            x, y, w, h = cv2.boundingRect(cnt)
                            digit = blackboard_gray[y:y + h, x:x + w]
                            # newImage = process_letter(digit)
                            pred_probab, pred_class = keras_predict(model1, digit)
                            print(pred_class, pred_probab)

                     pts = deque(maxlen=512)
                     blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
         cv2.putText(img, "Conv Network :  " + str(letter_count[pred_class]), (10, 470),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
         output.write(img)
         cv2.imshow("Frame", img)
         #cv2.imshow("Contours", thresh)
         if cv2.waitKey(25)&0XFF == ord('x'):
            break
            
    cap.release()
# close the already opened file
    output.release()
# close the window and de-allocate any associated memory usage
    cv2.destroyAllWindows()
        
except cv2.error as e:
     pass         
    